In [1]:
!pip install --upgrade pip
!pip install datasets evaluate rouge-score
!pip install hf_xet
!pip uninstall -y tokenizers
# !pip install -U accelerate
# !pip install accelerate==0.24.1
!pip install spacy sentence-transformers
!python -m spacy download en_core_web_sm
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'rouge-score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge-score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c5b55f54e2fbf5651c358e12992e51a3dc0746da8ccf7127693cd36c6f6405f0
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Succes

In [2]:
!pip uninstall -y transformers peft accelerate
!pip install -U accelerate
!pip install transformers==4.37.2
!pip install peft==0.10.0
!pip install accelerate==0.24.1

Found existing installation: transformers 4.51.3
Uninstalling transformers-4.51.3:
  Successfully uninstalled transformers-4.51.3
Found existing installation: peft 0.15.2
Uninstalling peft-0.15.2:
  Successfully uninstalled peft-0.15.2
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 95.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 136.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.1
    Uninstalling tokenizers-0.21.1:
      Successfully uninstalled tokenizers-0.21.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.4.1 requires transformers<5.0.0,>=4.41.0, but you have 

In [3]:
from datasets import load_dataset
from nltk.tokenize import sent_tokenize
import nltk
import random
import re
import spacy
from sentence_transformers import SentenceTransformer, util
nltk.download("punkt_tab")
nltk.download("punkt")

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
import re
import random
import spacy

nlp = spacy.load("en_core_web_sm")  # Load spaCy NER model

def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9.,;?!\'"()\[\]\s-]', '', text)
    return text.strip()

def get_distractors_from_context(context, correct_answer, max_distractors=2):
    doc = nlp(context)
    entities = list(set([ent.text for ent in doc.ents if ent.text.lower() != correct_answer.lower()]))
    random.shuffle(entities)
    return entities[:max_distractors] if entities else ["Paris", "London"]  # Fallback distractors

def mixed_format_question(example):
    context = clean_text(example["context"])
    question = example["question"]
    answers = example["answers"]["text"]
    correct_answer = answers[0] if answers else None

    if not correct_answer or correct_answer.strip() == "":
        return None

    question_type = random.choice(["general", "fill-in-the-blank", "multiple-choice"])

    # ================================
    # General Question
    # ================================
    if question_type == "general":
        input_text = f"Generate a general question from this context:\n{context}"
        target_text = question

    # ================================
    # Fill-in-the-Blank Question
    # ================================
    elif question_type == "fill-in-the-blank":
        sentences = re.split(r'(?<=[.!?]) +', context)
        target_sentence = next((s for s in sentences if re.search(rf"\b{re.escape(correct_answer)}\b", s)), None)

        if target_sentence:
            pattern = rf"\b{re.escape(correct_answer)}\b"
            blanked = re.sub(pattern, "_____", target_sentence)

            if blanked != target_sentence:
                input_text = f"Generate a fill-in-the-blank question from this context:\n{target_sentence}"
                target_text = blanked
            else:
                return None
        else:
            return None



    # ================================
    # Multiple Choice Question
    # ================================
    elif question_type == "multiple-choice":
        distractors = get_distractors_from_context(context, correct_answer)
        options = distractors + [correct_answer]
        random.shuffle(options)

        choices_str = "\n".join([f"{chr(65+i)}. {opt}" for i, opt in enumerate(options)])
        input_text = f"Generate a multiple-choice question from this context:\n{context}"
        target_text = f"{question}\n{choices_str}"

    if len(target_text.split()) < 5:
        return None

    return {
        "input_text": input_text,
        "target_text": target_text
    }


In [5]:
from datasets import load_dataset, DatasetDict
dataset = load_dataset("squad")

# Create 10% validation, 10% test, 80% train from the existing train set
full_train = dataset["train"].train_test_split(test_size=0.2, seed=42)
val_test = full_train["test"].train_test_split(test_size=0.5, seed=42)

# Now combine into one dataset dictionary
squad_split = DatasetDict({
    "train": full_train["train"],
    "validation": val_test["train"],  # 10%
    "test": val_test["test"]          # 10%
})

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
clean_train = squad_split["train"].map(mixed_format_question).filter(lambda x: x is not None)
clean_val = squad_split["validation"].map(mixed_format_question).filter(lambda x: x is not None)
clean_test = squad_split["test"].map(mixed_format_question).filter(lambda x: x is not None)


Map:   0%|          | 0/70079 [00:00<?, ? examples/s]

Filter:   0%|          | 0/66883 [00:00<?, ? examples/s]

Map:   0%|          | 0/8760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8416 [00:00<?, ? examples/s]

Map:   0%|          | 0/8760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/8366 [00:00<?, ? examples/s]

In [7]:
print(clean_train[0]["target_text"])
print(clean_val[0]["target_text"])
print(clean_test[0]["target_text"])
print(clean_train[2]["target_text"])

What was the cost of primary and secondary education under Gaddafi?
Which country dispatched regular troops to North America?
What was the last number Chopin gave to an opus?
A. Sonata
B. 65
C. 23
All in all, this makes Serbo-Croatian the only Slavic language to officially use both the _____ scripts, albeit the Latin version is more commonly used.


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def tokenize_function(example):
    model_inputs = tokenizer(
        example["input_text"], max_length=384, padding="max_length", truncation=True
    )
    labels = tokenizer(
        example["target_text"], max_length=64, padding="max_length", truncation=True
    )
    labels["input_ids"] = [
        (token if token != tokenizer.pad_token_id else -100)
        for token in labels["input_ids"]
    ]
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [9]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-large").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-large")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [10]:
# Tokenize each clean dataset
tokenized_train = clean_train.map(tokenize_function, batched=True)
tokenized_val = clean_val.map(tokenize_function, batched=True)
tokenized_test = clean_test.map(tokenize_function, batched=True)


Map:   0%|          | 0/66883 [00:00<?, ? examples/s]

Map:   0%|          | 0/8416 [00:00<?, ? examples/s]

Map:   0%|          | 0/8366 [00:00<?, ? examples/s]

In [11]:
columns_to_keep = ["input_ids", "attention_mask", "labels"]

tokenized_train = tokenized_train.remove_columns(
    [col for col in tokenized_train.column_names if col not in columns_to_keep])

tokenized_val = tokenized_val.remove_columns(
    [col for col in tokenized_val.column_names if col not in columns_to_keep])

tokenized_test = tokenized_test.remove_columns(
    [col for col in tokenized_test.column_names if col not in columns_to_keep])


In [12]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq
from transformers import EarlyStoppingCallback
training_args = TrainingArguments(
    output_dir="./flan-t5-large",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=3e-5,
    num_train_epochs=3,
    logging_steps=100,
    evaluation_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False
)


In [13]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


In [14]:
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: n-ogbi (n-ogbi-al-akhawayn-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss
1000,0.397900,0.351103
2000,0.361100,0.339483
3000,0.363300,0.335926
4000,0.370700,0.329890
5000,0.361900,0.327760
6000,0.357400,0.325919
7000,0.363400,0.324242
8000,0.371500,0.320506
9000,0.360000,0.322472
10000,0.357500,0.320768


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


TrainOutput(global_step=10000, training_loss=0.4858229814529419, metrics={'train_runtime': 16329.2661, 'train_samples_per_second': 12.288, 'train_steps_per_second': 3.072, 'total_flos': 6.914303852544e+16, 'train_loss': 0.4858229814529419, 'epoch': 0.6})

In [15]:
import torch
test_sample = clean_test.select(range(50))

predictions_test = []
references_test = []

model.eval()
for example in test_sample:
    input_text = example['input_text']
    reference = example['target_text']

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=128)

    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predictions_test.append(generated)
    references_test.append([reference])


In [25]:
import random

# Randomly select 10 indices (or any other number you prefer)
random_indices = random.sample(range(len(test_sample)), 50)

for idx, i in enumerate(random_indices):
    print(f"\nExample {idx+1}")
    print("Input:     ", test_sample[i]['input_text'])
    print("Reference: ", references_test[i][0])
    print("Prediction:", predictions_test[i])



Example 1
Input:      Generate a general question from this context:
If a capacitor is driven with a time-varying voltage that changes rapidly enough, at some frequency the polarization of the dielectric cannot follow the voltage. As an example of the origin of this mechanism, the internal microscopic dipoles contributing to the dielectric constant cannot move instantly, and so as frequency of an applied alternating voltage increases, the dipole response is limited and the dielectric constant diminishes. A changing dielectric constant with frequency is referred to as dielectric dispersion, and is governed by dielectric relaxation processes, such as Debye relaxation. Under transient conditions, the displacement field can be expressed as (see electric susceptibility)
Reference:  Under what condition might the polarization of the dielectric be unable to follow the voltage?
Prediction: What is the term for a changing dielectric constant with frequency?

Example 2
Input:      Generate a mu

In [17]:
import evaluate

bleu = evaluate.load("bleu")
rouge = evaluate.load("rouge")

In [18]:
import torch
# BLEU and ROUGE for validation set
val_sample = clean_val.select(range(50))

predictions_val = []
references_val = []

for example in val_sample:
    input_text = example['input_text']
    reference = example['target_text']

    inputs = tokenizer(input_text, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=128)

    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    predictions_val.append(generated)
    references_val.append([reference])

bleu_val = bleu.compute(predictions=predictions_val, references=references_val)
rouge_val = rouge.compute(predictions=predictions_val, references=[ref[0] for ref in references_val])

bleu_test = bleu.compute(predictions=predictions_test, references=references_test)
rouge_test = rouge.compute(predictions=predictions_test, references=[ref[0] for ref in references_test])


In [19]:
def print_eval_scores(name, bleu, rouge):
    print(f"\n {name} Set Evaluation (50 samples)")
    print(f"BLEU Score  : {bleu['bleu']:.4f}")
    print(f"ROUGE-L     : {rouge['rougeL']:.4f}")

print_eval_scores("Validation", bleu_val, rouge_val)
print_eval_scores("Test", bleu_test, rouge_test)



 Validation Set Evaluation (50 samples)
BLEU Score  : 0.4254
ROUGE-L     : 0.5047

 Test Set Evaluation (50 samples)
BLEU Score  : 0.5339
ROUGE-L     : 0.5292


In [20]:
import gradio as gr
import torch

def generate_question(paragraph, question_type):
    # Create prompt based on type
    prompt = f"Generate a {question_type} question from this context:\n{paragraph.strip()}"

    # Tokenize and generate
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(model.device)
    with torch.no_grad():
        output_ids = model.generate(**inputs, max_new_tokens=128)

    # Decode
    generated = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated

# Gradio interface
interface = gr.Interface(
    fn=generate_question,
    inputs=[
        gr.Textbox(lines=7, label="📘 Enter Educational Paragraph"),
        gr.Radio(["general", "fill-in-the-blank", "multiple-choice"], label="🧠 Question Type")
    ],
    outputs=gr.Textbox(label="✅ Generated Question"),
    title="LLM-Powered Test Question Generator",
    description="Paste a paragraph and choose the type of question to generate (general, fill-in-the-blank, or MCQ). Powered by fine-tuned Flan-T5-Large."
)

interface.launch(share=True)  # use share=True to get public link


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6bc20f166c46611836.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
